In [2]:

import geopandas
import fiona
import sys
from shapely.geometry import shape, GeometryCollection, Point, MultiPolygon, Polygon, asShape
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import shapefile as shp
from shapely import wkt
import matplotlib.pyplot as plt
import seaborn as sns



c:\Users\parae\anaconda3\envs\TEST\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

column = ["ground_amplitude", "flooding_risk", "danger_level", "ground_type", "altitude"
, "rain_day", "rain_hour", "longitude", "latitude", "processo_ocorrencia", "confirmado", "solicitacao_bairro", 
"solicitacao_descricao"]
df = pd.read_csv("ground_amplitude.csv", usecols=column)


def junction(row):
    if ((row['processo_ocorrencia'] == "Imoveis com Danos/Risco"  and ("barreira" in row['solicitacao_descricao'].lower()))
     or row['processo_ocorrencia'] == "Deslizamentos de Barreiras"
    ):
        return 1 #"ImoveisDanosRiscos_Deslizamentos"

    else:
        return  0#row['processo_ocorrencia']

"""
case "Imoveis com Danos/Risco":
            return "Deslizamento_ImoveisEmRisco"
        case "Deslizamentos de Barreiras":
            return "Deslizamento_ImoveisEmRisco"
"""


'\ncase "Imoveis com Danos/Risco":\n            return "Deslizamento_ImoveisEmRisco"\n        case "Deslizamentos de Barreiras":\n            return "Deslizamento_ImoveisEmRisco"\n'

In [4]:
df['filtered_ocorrencia'] = df.apply(lambda row: junction(row), axis=1)

df['filtered_ocorrencia'].head(20)


0     0
1     0
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     1
10    0
11    0
12    0
13    1
14    0
15    1
16    0
17    0
18    1
19    0
Name: filtered_ocorrencia, dtype: int64

In [5]:
#df = df.dropna()
#df.drop_duplicates()
df.pop("solicitacao_descricao")


df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5125 entries, 0 to 5124
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   solicitacao_bairro   5125 non-null   object 
 1   processo_ocorrencia  5125 non-null   object 
 2   confirmado           5125 non-null   int64  
 3   latitude             5125 non-null   float64
 4   longitude            5125 non-null   float64
 5   rain_hour            4855 non-null   float64
 6   rain_day             4850 non-null   float64
 7   altitude             4816 non-null   float64
 8   ground_type          5125 non-null   object 
 9   danger_level         5038 non-null   object 
 10  flooding_risk        5125 non-null   object 
 11  ground_amplitude     5038 non-null   object 
 12  filtered_ocorrencia  5125 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 520.6+ KB


In [13]:
le = LabelEncoder()

proc_ocorr = le.fit_transform(df['processo_ocorrencia'])
gro_type = le.fit_transform(df['ground_type'])
dan_level = le.fit_transform(df['danger_level'])
flo_risk = le.fit_transform(df['flooding_risk'])
gro_ampl = le.fit_transform(df['ground_amplitude'])
filt_ocorr = le.fit_transform(df['filtered_ocorrencia'])
soli_bair = le.fit_transform(df['solicitacao_bairro'])

df['processo_ocorrencia'] = proc_ocorr
df['ground_type'] = gro_type
df['danger_level'] = dan_level
df['flooding_risk'] = flo_risk
df['ground_amplitude'] = gro_ampl
df['filtered_ocorrencia'] = filt_ocorr
df['solicitacao_bairro'] = soli_bair
df.corr()


print(df[df.filtered_ocorrencia == 1].mean())
df[df.filtered_ocorrencia == 0].mean()

solicitacao_bairro     48.619638
processo_ocorrencia     6.633460
confirmado              0.788847
latitude               -8.038721
longitude             -34.932590
rain_hour               1.724899
rain_day               30.768511
altitude               45.338801
ground_type             4.000000
danger_level            1.551954
flooding_risk           2.863203
ground_amplitude        2.843184
filtered_ocorrencia     1.000000
dtype: float64


solicitacao_bairro     43.044268
processo_ocorrencia     7.160225
confirmado              0.000000
latitude               -8.052807
longitude             -34.924548
rain_hour               0.983641
rain_day               15.227079
altitude               30.069808
ground_type             4.011563
danger_level            1.359432
flooding_risk           1.905517
ground_amplitude        1.814998
filtered_ocorrencia     0.000000
dtype: float64

In [7]:
df.to_csv(r'C:\Users\parae\Documents\barreiras\barreiras\processing\slope\df.csv',
            index=False, header=True)